In [1]:
# 모듈 경로 추가
import sys
import os
sys.path.append(os.path.abspath('src'))

from predict_if import IsoForestPredictor

# 1. 예측기 초기화
detector = IsoForestPredictor(
    model_path='models/isolation_forest.joblib',
    scaler_path='models/scaler_if.joblib',
    result_dir='results/isolation_forest'
)

# 2. 예측 수행
target_csv = 'data/processed/1120_features_S1_yellow.csv'

# 파일명 기반으로 결과 파일명 자동 생성 예시
if os.path.exists(target_csv):
    base_name = os.path.basename(target_csv)
    output_name = f"pred_{base_name}"

    detector.predict(target_csv, output_filename=output_name)

📦 Loading Isolation Forest model...
✅ Model loaded successfully.

📂 Loading Data: data/processed/1120_features_S1_yellow.csv
🔍 Making Predictions...
   Total: 734, Anomalies: 695 (94.7%)
💾 Results saved to: results/isolation_forest\pred_1120_features_S1_yellow.csv
